### Importing the Libraries

In [ ]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col,dense_rank,avg,round
from snowflake.snowpark.window import Window

### Connect with Snowflake

In [ ]:
connection_params = {
    "account": "",
    "user": "",
    "password": "",
    "role": "",
    "warehouse": "",
    "database": "",
    "schema": ""
}

In [ ]:
try:
    session = Session.builder.configs(connection_params).create()
    print("Connected to Snowflake")
except Exception as e:
    print(e)

### Reading the Data from Snowflake

In [ ]:
location_df = session.sql("SELECT * FROM TELCO_PROJECT.TELCO_STAGING_TABLES.LOCATION")
status_analysis_df = session.sql("SELECT * FROM TELCO_PROJECT.TELCO_STAGING_TABLES.STATUS_ANALYSIS")
customer_df = session.sql("SELECT * FROM TELCO_PROJECT.TELCO_STAGING_TABLES.CUSTOMER")
online_service_df = session.sql("SELECT * FROM TELCO_PROJECT.TELCO_STAGING_TABLES.ONLINE_SERVICE")
payment_info_df = session.sql("SELECT * FROM TELCO_PROJECT.TELCO_STAGING_TABLES.PAYMENT_INFO")
service_option_df = session.sql("SELECT * FROM TELCO_PROJECT.TELCO_STAGING_TABLES.SERVICE_OPTION")

In [ ]:
location_df\
    .join(status_analysis_df, on = ["customer_id"], how = "inner")\
    .filter((col("CHURN_LABEL") == "Yes") & (col("CHURN_CATEGORY") != "Not Applicable"))\
    .groupBy(col("CITY"))\
    .agg(avg(col("CHURN_SCORE")).alias("avg_churn_score"))\
    .withColumn("drnk", dense_rank().over(Window.orderBy(col("avg_churn_score").desc())))\
    .withColumn("avg_churn_score", round(col("avg_churn_score"), 2))\
    .filter(col("drnk") <= 5)\
    .drop("drnk")